In [ ]:
# imports
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import text_dataset_from_directory

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import text_dataset_from_directory

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    
    def get_config(self):
        config = super().get_config()
        
        config.update(
            {
                'sequence_length': self.sequence_length,
                'input_dim': self.input_dim,
                'output_dim': self.output_dim
            }
        )
        return config

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        
        self.attention = layers.MultiHeadAttention(key_dim=embed_dim, nun_heads=num_heads)
        
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation='relu'),
                layers.Dense(embed_dim)
            ]
        )
        
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
            
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        
        proj_input = self.layernorm_1(inputs + attention_output)
        
        proj_output = self.dense_proj(proj_input)
        
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        
        config.update(
            {
                'embed_dim': self.embed_dim,
                'dense_dim': self.dense_dim,
                'num_heads': self.num_heads
            }
        )
        return config

In [4]:
class TrnasformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        
        self.supports_masking = True
        
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        
        self.attention_1 = layers.MultiHeadAttention(key_dim=embed_dim, num_heads=num_heads)
        self.attention_2 = layers.MultiHeadAttention(key_dim=embed_dim, num_heads=num_heads)
        
        self.dense_proj = keras.Sequential(
            [
                layer.Dense(dense_dim, activation='relu'),
                layer.Dense(embed_dim)
            ]
        )
        
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = inputs_shape[0], inputs_shape[1]
        
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, sequence_length, sequence_length))
        
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32)
            ], axis=0
        )
        
        return tf.tile(mask, mult)
    
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = get_causal_attention_mask(inputs)
        
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype='int32')
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
            
        attention_output_1 = self.attention_1(
            query=inputs,
            key=inputs,
            value=inputs,
            attention_mask=causal_mask
        )
        
        attention_ouput_1 = self.layernorm_1(inputs + attention_output_1)
        
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            key=encoder_outputs,
            value=encoder_outputs,
            attention_mask=padding_mask
        )
        
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        
        proj_output = self.dense_proj(attention_output_2)
        
        return self.layernorm_3(attention_output_2 + proj_output)
    
    def get_config(self):
        config = super().get_config()
        
        config.update(
            {
                'embed_dim': self.embed_dim,
                'dense_dim': self.dense_dim,
                'num_heads': self.num_heads
            }
        )
        return config

In [ ]:
strategy = tf.distribute.MirroredStrategy()

embed_dim = 256
dense_dim = 2046
num_heads = 7

with strategy.scope():
    encoder_inputs = keras.Input(shape=(None,), dtype='int64', name='english')
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    encoder_outputs = TransformerDecoder(embed_dim, dense_dim, num_heads)(x)
    
    decoder_inputs = keras.Input(shape=(None,), dtype='int64', name='spanish')
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
    x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
    
    x = layers.Dropout(0.5)(x)
    
    decoder_outputs = layers.Dense(vocab_size, activation='softmax')(x)
    
    transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    transformer.compile(
        optimizer='rmsprop',
        loss='sparce_categorical_crossentropy',
        metrics=['accuracy']

In [ ]:
decoder_inputs = keras.Input(shape=(None,), dtype='int64', name='english')
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_inputs)

x = layers.Dropout(0.5)(x)
decoder_output = layers.Dense(vocab_size, acitivation='softmax')(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_output)

transformer.compile(
    optimizer='rmsprop',
    loss='sparce_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
strategy = tf.distribute.MirroredStrategy()

embed_dim = 256
dense_dim = 2046
num_heads = 7

with strategy.scope():
    encoder_inputs = keras.Input(shape(None,), dtype='int64', name='inglish')
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
    encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    
    decoder_input = keras.Input(shape=(None,), dtype='int64', name='spanish')
    x = PositionalEmbedding(sequence_length, vacab_size, embed_dim)(decoder_input)
    x = TransformerDecoder(embed_dim, dense_dim, num_heads)(decoder_inputs, encoder_output)
    
    x = layers.Dropout(0.5)(x)
    
    decoder_output = layer.Dense(vocab_size, activation='softmax')
    
    transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_output)
    
    transformer.compile(
        optimizer='rmsprop',
        loss='sparse_category_crossentropy',
        metrics=['accuracy']
    )

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    
    encoder_inputs = keras.Input(shape=(None, ), dtype='int64', name='english')
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim

In [ ]:
class Node(object):
    def __init__(self, val):
        self.val = val
        self.next = None
        
    def get_data(self):
        return self.val
    
    def set_data(self, val):
        self.val = val
        
    def get_next(self):
        return self.next
    
    def set_next(self, next):
        self.next = next
        
     
        
class LinkedList(object):
    def __init__(self, head=None):
        self.head = head
        self.count = 0
    
    def get_count(self):
        return self.count
    
    def is_empty(self):
        return self.head == None
    
    def insert(self, data):
        
        new_node = Node(data)
        new_node.set_next(self.head)
        
        self.head = new_node        
        self.count += 1
        
    def find(self, val):
        item = self.head
        
        while item is not None:
            if item.get_data() == val:
                return item
            else:
                item = item.get_next()
        
        return None
    
    def display(self):
        current = self.head
        while current:
            print(current.val, end=" -> ")
            current = current.next
            
        print("None")
    
    
    def remove(self, item):
        current = self.head
        previous = None
        
        while current is not None:
            if current.get_data() == item:
                break
                
            previous = current
            current = current.get_next()
            
        if current is None: # item not in the list
            raise ValueError(f"{item} is not in the list")
            
        if previous is None: # the item is the head
            self.head = current.get_next()
        else: # just remove the item from the list
            previous.set_next(current.get_next())
        
        self.count -= 1   
        
    

In [6]:
i = 0
while i <= 4:
    if i == 2:
        print(f"break= {i}")
        break
    print(f"still inside the loop {i}")
    
    i += 1

print("outside the loop")

still inside the loop 0
still inside the loop 1
break= 2
outside the loop


In [10]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None



class BinarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, value):
        if self.root is None:
            self.root = Node(value)
        else:
            self._insert(value, self.root)

    def _insert(self, value, current_node):
        if value < current_node.value:
            if current_node.left is None:
                current_node.left = Node(value)
            else:
                self._insert(value, current_node.left)
        elif value > current_node.value:
            if current_node.right is None:
                current_node.right = Node(value)
            else:
                self._insert(value, current_node.right)
        else:
            print("Value already exists in the tree!")

    def search(self, value):
        return self._search(value, self.root)

    def _search(self, value, current_node):
        if current_node is None:
            return False
        if value == current_node.value:
            return True
        elif value < current_node.value:
            return self._search(value, current_node.left)
        else:
            return self._search(value, current_node.right)

    def delete(self, value):
        self.root = self._delete(value, self.root)

    def _delete(self, value, current_node):
        if current_node is None:
            return current_node

        if value < current_node.value:
            current_node.left = self._delete(value, current_node.left)
        elif value > current_node.value:
            current_node.right = self._delete(value, current_node.right)
        else:
            if current_node.left is None:
                return current_node.right
            elif current_node.right is None:
                return current_node.left

            temp = self._find_min(current_node.right)
            current_node.value = temp.value
            current_node.right = self._delete(temp.value, current_node.right)
        
        return current_node

    def _find_min(self, node):
        current = node
        while current.left is not None:
            current = current.left
        return current

    def inorder_traversal(self):
        return self._inorder_traversal(self.root, [])

    def _inorder_traversal(self, node, values):
        if node:
            self._inorder_traversal(node.left, values)
            values.append(node.value)
            self._inorder_traversal(node.right, values)
        return values

# Example usage
bst = BinarySearchTree()
bst.insert(10)
bst.insert(5)
bst.insert(15)
bst.insert(2)
bst.insert(7)
bst.insert(12)
bst.insert(17)

print("In-order Traversal:", bst.inorder_traversal())  # [2, 5, 7, 10, 12, 15, 17]

print("Search 7:", bst.search(7))  # True
print("Search 20:", bst.search(20))  # False

bst.delete(10)
print("In-order Traversal after deleting 10:", bst.inorder_traversal())  # [2, 5, 7, 12, 15, 17]


In-order Traversal: [2, 5, 7, 10, 12, 15, 17]
Search 7: True
Search 20: False
In-order Traversal after deleting 10: [2, 5, 7, 12, 15, 17]


In [2]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class BinarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, value):
        new_node = Node(value)
        if self.root is None:
            self.root = new_node
        else:
            self._insert(new_node, self.root)

    def _insert(self, new_node, current_node):
        if new_node.value < current_node.value:
            if current_node.left is None:
                current_node.left = new_node
            else:
                self._insert(new_node, current_node.left)
        elif new_node.value > current_node.value:
            if current_node.right is None:
                current_node.right = new_node
            else:
                self._insert(new_node, current_node.right)
        else:
            print("Value already exists in the tree!")

    def search(self, value):
        if self.root is None:
            return False
        else:
            return self._search(value, self.root)

    def _search(self, value, current_node):
        if value == current_node.value:
            return True
        elif value < current_node.value:
            if current_node.left is None:
                return False
            else:
                return self._search(value, current_node.left)
        else:
            if current_node.right is None:
                return False
            else:
                return self._search(value, current_node.right)

    def print_tree(self):
        if self.root is not None:
            self._print_tree(self.root)

    def _print_tree(self, current_node):
        if current_node is not None:
            self._print_tree(current_node.left)
            print(current_node.value)
            self._print_tree(current_node.right)

# Example usage:
bst = BinarySearchTree()
bst.insert(5)
bst.insert(3)
bst.insert(7)
bst.insert(2)
bst.insert(4)
bst.insert(6)
bst.insert(8)

print(bst.search(5))  # True
print(bst.search(10))  # False

bst.print_tree()


True
False
2
3
4
5
6
7
8


In [9]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class BinarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, value):
        new_node = Node(value)
        if self.root is None:
            self.root = new_node
        else:
            self._insert(new_node, self.root)

    def _insert(self, new_node, current_node):
        if new_node.value < current_node.value:
            if current_node.left is None:
                current_node.left = new_node
            else:
                self._insert(new_node, current_node.left)
        elif new_node.value > current_node.value:
            if current_node.right is None:
                current_node.right = new_node
            else:
                self._insert(new_node, current_node.right)
        else:
            print("Value already exists in the tree!")

    def search(self, value):
        if self.root is None:
            return False
        else:
            return self._search(value, self.root)

    def _search(self, value, current_node):
        if value == current_node.value:
            return True
        elif value < current_node.value:
            if current_node.left is None:
                return False
            else:
                return self._search(value, current_node.left)
        else:
            if current_node.right is None:
                return False
            else:
                return self._search(value, current_node.right)

    def print_tree(self):
        if self.root is not None:
            self._print_tree(self.root)

    def _print_tree(self, current_node):
        if current_node is not None:
            self._print_tree(current_node.left)
            print(current_node.value)
            self._print_tree(current_node.right)

    def print_tree_structure(self):
        lines, *_ = self._display_aux(self.root)
        for line in lines:
            print(line)

    def _display_aux(self, node):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if node.right is None and node.left is None:
            line = f'{node.value}'
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if node.right is None:
            lines, n, p, x = self._display_aux(node.left)
            s = f'{node.value}'
            u = len(s)
            first_line = s + x * ' ' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Only right child.
        if node.left is None:
            lines, n, p, x = self._display_aux(node.right)
            s = f'{node.value}'
            u = len(s)
            first_line = s + x * ' ' + (n - x) * ' '
            second_line = (u + x) * ' ' + '/' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self._display_aux(node.left)
        right, m, q, y = self._display_aux(node.right)
        s = f'{node.value}'
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

# Example usage:
bst = BinarySearchTree()
bst.insert(5)
bst.insert(3)
bst.insert(7)
bst.insert(2)
bst.insert(4)
bst.insert(6)
bst.insert(8)

print(bst.search(5))  # True
print(bst.search(10))  # False

bst.print_tree_structure()


True
False
  _5_  
 /   \ 
 3   7 
/ \ / \
2 4 6 8
